In [2]:
import os
import tensorflow as tf
from tensorflow.core.protobuf import saver_pb2
import driving_data
import model

In [3]:
LOGDIR = './save'

In [4]:
sess = tf.InteractiveSession()

In [5]:
L2NormConst = 0.001

In [6]:
train_vars = tf.trainable_variables()

In [10]:
loss = tf.reduce_mean(tf.square(tf.subtract(model.y_, model.y))) + tf.add_n([tf.nn.l2_loss(v) for v in train_vars]) * L2NormConst
train_step = tf.train.AdamOptimizer(1e-3).minimize(loss)
sess.run(tf.initialize_all_variables())

Instructions for updating:
Use `tf.global_variables_initializer` instead.


In [11]:
tf.summary.scalar("loss", loss)
# merge all summaries into a single op
merged_summary_op =  tf.summary.merge_all()

In [12]:
saver = tf.train.Saver(write_version = saver_pb2.SaverDef.V1)

In [13]:
logs_path = './logs'
summary_writer = tf.summary.FileWriter(logs_path, graph=tf.get_default_graph())

In [14]:
epochs = 30
batch_size = 100

In [15]:
for epoch in range(epochs):
  for i in range(int(driving_data.num_images/batch_size)):
    xs, ys = driving_data.LoadTrainBatch(batch_size)
    train_step.run(feed_dict={model.x: xs, model.y_: ys, model.keep_prob: 0.5})
    if i % 10 == 0:
      xs, ys = driving_data.LoadValBatch(batch_size)
      loss_value = loss.eval(feed_dict={model.x:xs, model.y_: ys, model.keep_prob: 0.5})
      print("Epoch: %d, Step: %d, Loss: %g" % (epoch, epoch * batch_size + i, loss_value))

    # write logs at every iteration
    summary = merged_summary_op.eval(feed_dict={model.x:xs, model.y_: ys, model.keep_prob: 0.5})
    summary_writer.add_summary(summary, epoch * driving_data.num_images/batch_size + i)
    if i % batch_size == 0:
      if not os.path.exists(LOGDIR):
        os.makedirs(LOGDIR)
      checkpoint_path = os.path.join(LOGDIR, "model.ckpt")
      filename = saver.save(sess, checkpoint_path)
  print("Model saved in file: %s" % filename)

Epoch: 0, Step: 0, Loss: 7.14837
Epoch: 0, Step: 10, Loss: 5.67808
Epoch: 0, Step: 20, Loss: 5.30173
Epoch: 0, Step: 30, Loss: 4.96623
Epoch: 0, Step: 40, Loss: 4.92185
Epoch: 0, Step: 50, Loss: 4.34956
Epoch: 0, Step: 60, Loss: 4.2462
Epoch: 0, Step: 70, Loss: 4.218
Epoch: 0, Step: 80, Loss: 3.88259
Epoch: 0, Step: 90, Loss: 3.50505
Epoch: 0, Step: 100, Loss: 3.31965
Epoch: 0, Step: 110, Loss: 3.13987
Epoch: 0, Step: 120, Loss: 3.02384
Epoch: 0, Step: 130, Loss: 3.18539
Epoch: 0, Step: 140, Loss: 3.54065
Epoch: 0, Step: 150, Loss: 2.95981
Epoch: 0, Step: 160, Loss: 3.45093
Epoch: 0, Step: 170, Loss: 2.76323
Epoch: 0, Step: 180, Loss: 2.95903
Epoch: 0, Step: 190, Loss: 2.62989
Epoch: 0, Step: 200, Loss: 2.23499
Epoch: 0, Step: 210, Loss: 2.16127
Epoch: 0, Step: 220, Loss: 2.14044
Epoch: 0, Step: 230, Loss: 2.16618
Epoch: 0, Step: 240, Loss: 2.98621
Epoch: 0, Step: 250, Loss: 2.10058
Epoch: 0, Step: 260, Loss: 4.17056
Epoch: 0, Step: 270, Loss: 2.36966
Epoch: 0, Step: 280, Loss: 2.87085

In [16]:
print("Run the command line:\n" \
          "--> tensorboard --logdir=./logs " \
          "\nThen open http://0.0.0.0:6006/ into your web browser")

Run the command line:
--> tensorboard --logdir=./logs 
Then open http://0.0.0.0:6006/ into your web browser
